In [1]:
import pandas as pd
import re
from itertools import combinations

In [2]:
df = pd.read_csv('FriendsReportCsv_2025-04-22.csv')
users = pd.read_csv('UserCreatedReport_2025-04-23 09_57.csv')

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_7594/1450988638.py:2: DtypeWarning: Columns (14,15,19,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('UserCreatedReport_2025-04-23 09_57.csv')


In [3]:
df['Full Name'] = df['First name'] + " " + df['Last name']
df = df[['assign using  ID number', 'Full Name', 'FriendsList']]
df

,assign using ID number,Full Name,FriendsList
0,15394992,גוסטבו שנייברג,207877705 אור שנייברג
1,301137543,בתאל הס,"221890023 צור הס , 226599231 ראם הס , 2230067..."
2,200500007,נועה בודנקין,"227855459 ינאי משה , 046221909 אסי סיגטי"
3,217743160,שניר גרונוולד,208068189 איל שרמן
4,032531709,דן ורנר,344639463 אביתר ורנר
...,...,...,...
3706,300957354,אבי בן הלל,"224953729 עמית בן הלל אבנון , 226604114 יהלי ב..."
3707,21604350,Mattan Shpaier,"221964570 אופיר שפייר , 343278248 זהר שפייר , ..."
3708,021348081,ערן טוכמן,"036501393 שמרית טוכמן , 225041631 ארז טוכמן"
3709,043127976,מיכל קרני,"219147626 שלו קרני , 033039603 איתמר קרני"


## Create pairs of full name and friendslist with ids:

In [5]:
pair_df = []

for _, row in df.iterrows():
    # extracting the values so it will be easier to use
    person_id = row['assign using  ID number']
    person_name = row['Full Name']
    friends_raw = row['FriendsList']

    friends = [] 
    # extract the id and name pairs
    # [ ] = match any one character inside the brackets
    # \w = matches any word character (letters, digits, or underscores)
    # @ = matches @
    # . = matches a dot
    # \+ = matches a plus sign
    # \- = matches a - sign
    # + = one or more occurrences of the preceding character or group
    # ( ) = captures this group as the first match
    
    # \s = matches any whitespace character
    
    # [ ] = match any one character inside the brackets
    # -א-ת = matches any Hebrew character
    # A-Za-z = matches any English alphabet character

    friends = re.findall(r'([\w@.+\-]+)\s+([-א-תA-Za-z\s]+)', friends_raw)

    # append it to the table
    for friend_id, friend_name in friends:
        pair_df.append({
        'Person ID' : person_id,
        'Person Name' : person_name,
        'Friend ID' : friend_id,
        'Friend Name' : friend_name.strip()   # strip - cleanup spaces
        })

    # append it to the table
    for (person1_id, person1_name), (friend1_id, friend1_name) in combinations(friends, 2):
        pair_df.append({
            'Person ID': person1_id,
            'Person Name': person1_name.strip(), # strip - cleanup spaces
            'Friend ID': friend1_id,
            'Friend Name': friend1_name.strip() # strip - cleanup spaces
        })

pair_df = pd.DataFrame(pair_df)

pair_df.drop_duplicates()

pair_df

,Person ID,Person Name,Friend ID,Friend Name
0,15394992,גוסטבו שנייברג,207877705,אור שנייברג
1,301137543,בתאל הס,221890023,צור הס
2,301137543,בתאל הס,226599231,ראם הס
3,301137543,בתאל הס,223006743,תמר הס
4,301137543,בתאל הס,221072838,שקד הס
...,...,...,...,...
16103,036501393,שמרית טוכמן,225041631,ארז טוכמן
16104,043127976,מיכל קרני,219147626,שלו קרני
16105,043127976,מיכל קרני,033039603,איתמר קרני
16106,219147626,שלו קרני,033039603,איתמר קרני


## Check if all went well and the data is correct:

In [7]:
check = df[~df['Full Name'].isin(pair_df['Person Name'])]
check

,assign using ID number,Full Name,FriendsList


## Add the age column to users based on the birthdate column:

In [9]:
from datetime import date

# convert the string to datetime
users['bithdate'] = pd.to_datetime(users['birthdate'], errors='coerce')

# extract the year from the bithdate column
users['Year'] = users['bithdate'].dt.year

# getting todays year
todays_year = date.today() 
todays_year = todays_year.year

# calculation of the age
users['Age'] = todays_year - users['Year']

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_7594/3710153648.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  users['bithdate'] = pd.to_datetime(users['birthdate'], errors='coerce')


## Adding the age column for both Person Name and Friend Name:

In [11]:
# ensure identifier in users is unique
users = users.drop_duplicates(subset=['identifier'])

# add Age column for Person ID
pair_with_age = pair_df.merge(users[['identifier', 'Age']], 
                     left_on='Person ID', 
                     right_on='identifier', 
                     how='left')

# rename 
pair_with_age.rename(columns={'Age': 'Person Age'}, inplace=True)

# add Age column for Friend ID
pair_with_age = pair_with_age.merge(users[['identifier', 'Age']], 
                  left_on='Friend ID', 
                  right_on='identifier', 
                  how='left')

# rename
pair_with_age.rename(columns={'Age': 'Friend Age'}, inplace=True)

# drop duplicates
pair_with_age.drop(columns=['identifier_x', 'identifier_y'], inplace=True)

pair_with_age = pair_with_age[['Person ID', 'Person Name', 'Person Age', 'Friend ID', 'Friend Name', 'Friend Age']]

pair_with_age

,Person ID,Person Name,Person Age,Friend ID,Friend Name,Friend Age
0,15394992,גוסטבו שנייברג,62.0,207877705,אור שנייברג,26.0
1,301137543,בתאל הס,38.0,221890023,צור הס,12.0
2,301137543,בתאל הס,38.0,226599231,ראם הס,7.0
3,301137543,בתאל הס,38.0,223006743,תמר הס,10.0
4,301137543,בתאל הס,38.0,221072838,שקד הס,13.0
...,...,...,...,...,...,...
16103,036501393,שמרית טוכמן,46.0,225041631,ארז טוכמן,7.0
16104,043127976,מיכל קרני,44.0,219147626,שלו קרני,15.0
16105,043127976,מיכל קרני,44.0,033039603,איתמר קרני,49.0
16106,219147626,שלו קרני,15.0,033039603,איתמר קרני,49.0


## Drop rows based on age conditions:

In [13]:
# drop the rows when ages are greater than 70 or are equals to each other
pair_cond = pair_with_age[
    ~((pair_with_age['Person Age'] == pair_with_age['Friend Age']) |
    ((pair_with_age['Person Age'] >= 70) | (pair_with_age['Friend Age'] >= 70)))
]

# minimum parent age
threshold = 24 

# drop rows where the age difference is less than the threshold - they cannot be a parent-child pair
pair_cond = pair_cond[~(abs(pair_cond['Person Age'] - pair_cond['Friend Age'] < threshold))]


# drop rows when when both ages are greater than 18
pair_cond = pair_cond[~((pair_cond['Person Age'] > 18) & (pair_cond['Friend Age'] > 18))]

# drop occurance of nan values in ages
pair_cond = pair_cond[~((pair_cond['Person Age'].isna()) | (pair_cond['Friend Age'].isna()))]

pair_cond

,Person ID,Person Name,Person Age,Friend ID,Friend Name,Friend Age
1,301137543,בתאל הס,38.0,221890023,צור הס,12.0
2,301137543,בתאל הס,38.0,226599231,ראם הס,7.0
3,301137543,בתאל הס,38.0,223006743,תמר הס,10.0
4,301137543,בתאל הס,38.0,221072838,שקד הס,13.0
5,301137543,בתאל הס,38.0,343280210,גפן הס,8.0
...,...,...,...,...,...,...
16096,21604350,Mattan Shpaier,40.0,343278248,זהר שפייר,8.0
16097,21604350,Mattan Shpaier,40.0,234630978,אורי שפייר,4.0
16102,021348081,ערן טוכמן,46.0,225041631,ארז טוכמן,7.0
16103,036501393,שמרית טוכמן,46.0,225041631,ארז טוכמן,7.0


## Drop rows based on last name:

In [29]:
# add Age column for Person ID
final = pair_cond.merge(users[['identifier', 'lastname']], 
                     left_on='Person ID', 
                     right_on='identifier', 
                     how='left')

# rename 
final.rename(columns={'lastname': 'Person Last Name'}, inplace=True)

# add Age column for Friend ID
final = final.merge(users[['identifier', 'lastname']], 
                  left_on='Friend ID', 
                  right_on='identifier', 
                  how='left')

# rename
final.rename(columns={'lastname': 'Friend Last Name'}, inplace=True)

# drop rows when the last name is not equal
final = final[~(final['Person Last Name'] != final['Friend Last Name'])]

parent_child = final[['Person ID', 'Person Name', 'Person Age', 'Friend ID', 'Friend Name', 'Friend Age']]

# drop rows when the name contains only one word
# str.strip() - removes leading and trailing whitespace from the names
# str.count(r'\s') - counts the number of spaces in the name. If the count is 0 - the name contains only one word
parent_child = parent_child[
    (parent_child['Person Name'].str.strip().str.count(r'\s') > 0) &  
    (parent_child['Friend Name'].str.strip().str.count(r'\s') > 0)
]

parent_child

,Person ID,Person Name,Person Age,Friend ID,Friend Name,Friend Age
0,301137543,בתאל הס,38.0,221890023,צור הס,12.0
1,301137543,בתאל הס,38.0,226599231,ראם הס,7.0
2,301137543,בתאל הס,38.0,223006743,תמר הס,10.0
3,301137543,בתאל הס,38.0,221072838,שקד הס,13.0
4,301137543,בתאל הס,38.0,343280210,גפן הס,8.0
...,...,...,...,...,...,...
4947,038086252,נטע שורץ,40.0,340681048,הלל שורץ,10.0
4949,037720299,ערן שורץ,42.0,340681048,הלל שורץ,10.0
4955,021348081,ערן טוכמן,46.0,225041631,ארז טוכמן,7.0
4956,036501393,שמרית טוכמן,46.0,225041631,ארז טוכמן,7.0


In [31]:
parent_child.to_excel('parent_child_candidates.xlsx')